# Tutorial Goal

This tutorial aims to show how to **configure and run** a **synthetic workload** in Hikey960 using the **wlgen module** provided by LISA.

# Configure logging

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2017-07-14 15:01:16,922 INFO    : root         : Using LISA logging configuration:
2017-07-14 15:01:16,924 INFO    : root         :   /data/lisa_remote/lisa/logging.conf


In [2]:
# Execute this cell to enabled devlib debugging statements
logging.getLogger().setLevel(logging.DEBUG)

In [3]:
# Other python modules required by this notebook
import json
import os

# Test environment setup

In [4]:
# Setup a target configuration
conf = {
    
    # Target is localhost
    "platform"    : 'android',
    "board"       : "hikey960",
    
    # Login credentials
    #"host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",

    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # Comment the following line to force rt-app calibration on your target
     "rtapp-calib" : {
        "0": 301, "1": 301, "2": 301, "3": 301, "4": 226, "5": 226, "6": 226, "7": 226
     },
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
             "sched_switch",
             "cpu_frequency"
         ],
         "buffsize" : 10240
    },

    # Where results are collected
    "results_dir" : "WlgenExample_hikey960",
    
    # Devlib modules we'll need
    "modules": ["cpufreq"]
}

In [6]:
# Support to access the remote target
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(conf)
target = te.target

2017-07-14 15:01:51,363 INFO    : TestEnv      : Using base path: /data/lisa_remote/lisa
2017-07-14 15:01:51,369 INFO    : TestEnv      : Loading custom (inline) target configuration
2017-07-14 15:01:51,372 INFO    : TestEnv      : External tools using:
2017-07-14 15:01:51,374 INFO    : TestEnv      :    ANDROID_HOME: /home/zhifei/zyang/android-sdk-linux
2017-07-14 15:01:51,375 INFO    : TestEnv      :    CATAPULT_HOME: /data/lisa_remote/lisa/tools/catapult
2017-07-14 15:01:51,377 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
2017-07-14 15:01:51,381 INFO    : TestEnv      : Connecting Android target [DEFAULT]
2017-07-14 15:01:51,382 INFO    : TestEnv      : Connection settings:
2017-07-14 15:01:51,383 INFO    : TestEnv      :    None
2017-07-14 15:01:52,299 INFO    : TestEnv      : Initializing target workdir:
2017-07-14 15:01:52,305 INFO    : TestEnv      :    /data/local/tmp/devlib-target
2017-07-14 15:01:55,374 INFO    : TestEnv      : Topology:
2017-

## Workload execution utility

In [7]:
#
# if Hikey960 board has AEP or ACME connection, set below variable as 1
# or else set it as 0
#
hikey960_has_energy_meter = 0


def execute(te, wload, res_dir):
    
    logging.info('# Create results folder for this execution')
    !mkdir {res_dir}
    
    logging.info('# Setup FTrace')
    te.ftrace.start()

    if hikey960_has_energy_meter:
        logging.info('## Start energy sampling')
        te.emeter.reset()

    logging.info('### Start RTApp execution')
    wload.run(out_dir=res_dir)

    if hikey960_has_energy_meter:
        logging.info('## Read energy consumption: %s/energy.json', res_dir)
        nrg_report = te.emeter.report(out_dir=res_dir)
    else:
        nrg_report = None

    logging.info('# Stop FTrace')
    te.ftrace.stop()

    trace_file = os.path.join(res_dir, 'trace.dat')
    logging.info('# Save FTrace: %s', trace_file)
    te.ftrace.get_trace(trace_file)

    logging.info('# Save platform description: %s/platform.json', res_dir)
    plt, plt_file = te.platform_dump(res_dir)
    
    logging.info('# Report collected data:')
    logging.info('   %s', res_dir)
    !ls -la {res_dir}
    
    return nrg_report, plt, plt_file, trace_file

# Single task RTApp workload

## 1) creation

In [8]:
# Support to configure and run RTApp based workloads
from wlgen import RTA

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp_name = 'example1'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

2017-07-14 15:02:30,017 INFO    : Workload     : Setup new workload example1


## 2) configuration

In [9]:
# RTApp configurator for generation of PERIODIC tasks
from wlgen import Periodic 

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 100, # period [ms]
            duty_cycle_pct =  20, # duty cycle [%]
            duration_s     =  5,  # duration [s]
            delay_s        =  0,  # start after that delay [s]
            sched          = {    # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 130,
            },
            cpus           =      # pinned on first online CPU
                              str(target.list_online_cpus()[0])
            # ADD OTHER PARAMETERS
        ).get(),

    },
    
);

2017-07-14 15:02:31,836 INFO    : Workload     : Workload duration defined by longest task
2017-07-14 15:02:31,839 INFO    : Workload     : Default policy: SCHED_OTHER
2017-07-14 15:02:31,845 INFO    : Workload     : ------------------------
2017-07-14 15:02:31,846 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER', 'priotity': 130}
2017-07-14 15:02:31,852 INFO    : Workload     :  | calibration CPU: 4
2017-07-14 15:02:31,857 INFO    : Workload     :  | loops count: 1
2017-07-14 15:02:31,858 INFO    : Workload     :  | CPUs affinity: 0
2017-07-14 15:02:31,859 INFO    : Workload     : + phase_000001: duration 5.000000 [s] (50 loops)
2017-07-14 15:02:31,864 INFO    : Workload     : |  period   100000 [us], duty_cycle  20 %
2017-07-14 15:02:31,869 INFO    : Workload     : |  run_time  20000 [us], sleep_time  80000 [us]


In [10]:
# Inspect the JSON file used to run the application
with open('./{}_00.json'.format(rtapp_name), 'r') as fh:
    rtapp_json = json.load(fh)
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)

2017-07-14 15:02:35,846 INFO    : root         : Generated RTApp JSON file:


{
    "global": {
        "calibration": 226, 
        "default_policy": "SCHED_OTHER", 
        "duration": -1, 
        "logdir": "/data/local/tmp/devlib-target"
    }, 
    "tasks": {
        "task_p20": {
            "cpus": [
                0
            ], 
            "loop": 1, 
            "phases": {
                "p000001": {
                    "loop": 50, 
                    "run": 20000, 
                    "timer": {
                        "period": 100000, 
                        "ref": "task_p20"
                    }
                }
            }, 
            "policy": "SCHED_OTHER", 
            "priotity": 130
        }
    }
}


## 3) execution

In [11]:
res_dir = os.path.join(te.res_dir, rtapp_name)
nrg_report, plt, plt_file, trace_file = execute(te, rtapp, res_dir)

2017-07-14 15:03:18,095 INFO    : root         : # Create results folder for this execution
2017-07-14 15:03:18,215 INFO    : root         : # Setup FTrace
2017-07-14 15:03:20,617 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:03:20,740 DEBUG   : FtraceCollector : Trace CPUIdle states
2017-07-14 15:03:20,833 INFO    : root         : ### Start RTApp execution
2017-07-14 15:03:20,837 INFO    : Workload     : Workload execution START:
2017-07-14 15:03:20,840 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/example1_00.json 2>&1


2017-07-14 15:03:26,186 INFO    : root         : # Stop FTrace
2017-07-14 15:03:26,193 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:03:26,409 INFO    : root         : # Save FTrace: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example1/trace.dat
2017-07-14 15:03:28,315 INFO    : root         : # Save platform description: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example1/platform.json
2017-07-14 15:03:28,326 INFO    : root         : # Report collected data:
2017-07-14 15:03:28,331 INFO    : root         :    /data/lisa_remote/lisa/results/WlgenExample_hikey960/example1


total 4492
drwxrwxr-x 2 zhifei zhifei    4096 7月  14 15:03 .
drwxrwxr-x 3 zhifei zhifei    4096 7月  14 15:03 ..
-rw-r--r-- 1 zhifei zhifei     654 7月  14 15:03 example1_00.json
-rw-rw-r-- 1 zhifei zhifei     170 7月  14 15:03 output.log
-rw-rw-r-- 1 zhifei zhifei    1528 7月  14 15:03 platform.json
-rw-r--r-- 1 zhifei zhifei    6360 7月  14 15:03 rt-app-task_p20-0.log
-rw-r--r-- 1 zhifei zhifei 4571136 7月  14 15:03 trace.dat


## 4) Check collected data

In [12]:
# Dump the energy measured for the LITTLE and big clusters

if hikey960_has_energy_meter:
    logging.info('Energy: %s', nrg_report.report_file)
    print json.dumps(nrg_report.channels, indent=4, sort_keys=True)
else:
    print "No energy result because there is no energy meter"

No energy result because there is no energy meter


In [13]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
print json.dumps(plt, indent=4, sort_keys=True)

2017-07-14 15:03:46,203 INFO    : root         : Platform description: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example1/platform.json


{
    "abi": "arm64", 
    "clusters": {
        "big": [
            4, 
            5, 
            6, 
            7
        ], 
        "little": [
            0, 
            1, 
            2, 
            3
        ]
    }, 
    "cpus_count": 8, 
    "freqs": {
        "big": [
            903000, 
            1421000, 
            1805000, 
            2112000, 
            2362000
        ], 
        "little": [
            533000, 
            999000, 
            1402000, 
            1709000, 
            1844000
        ]
    }, 
    "kernel": {
        "major": 4, 
        "minor": 59, 
        "parts": [
            4, 
            4, 
            59
        ], 
        "rc": null, 
        "release": "4.4.59-02536-g679a543", 
        "sha1": "679a543", 
        "version": "2507 SMP PREEMPT Tue Apr 11 09:52:10 PDT 2017", 
        "version_number": 4
    }, 
    "nrg_model": {
        "big": {
            "cluster": {
                "nrg_max": 118
            }, 
       

## 5) trace inspection

In [14]:
!kernelshark {trace_file} 2>/dev/null

version = 6


# Workload composition using RTApp

## 1) creation

In [15]:
# Support to configure and run RTApp based workloads
from wlgen import RTA

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp_name = 'example2'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

2017-07-14 15:03:58,018 INFO    : Workload     : Setup new workload example2


## 2) configuration

In [16]:
# RTApp configurator for generation of PERIODIC tasks
from wlgen import Periodic, Ramp

# Light workload
light  = Periodic(duty_cycle_pct=10, duration_s=1.0, period_ms= 10,
              cpus=str(target.bl.bigs_online[0]))
# Ramp workload
ramp   = Ramp(start_pct=10, end_pct=90, delta_pct=20, time_s=1, period_ms=50)
# Heavy workload
heavy  = Periodic(duty_cycle_pct=90, duration_s=0.1, period_ms=100)

# Composed workload
lrh_task = light + ramp + heavy

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_ramp': lrh_task.get() 

    },
    
);

2017-07-14 15:03:59,495 INFO    : Workload     : Workload duration defined by longest task
2017-07-14 15:03:59,499 INFO    : Workload     : Default policy: SCHED_OTHER
2017-07-14 15:03:59,500 INFO    : Workload     : ------------------------
2017-07-14 15:03:59,504 INFO    : Workload     : task [task_ramp], sched: using default policy
2017-07-14 15:03:59,509 INFO    : Workload     :  | calibration CPU: 4
2017-07-14 15:03:59,512 INFO    : Workload     :  | loops count: 1
2017-07-14 15:03:59,513 INFO    : Workload     :  | CPUs affinity: 4
2017-07-14 15:03:59,517 INFO    : Workload     : + phase_000001: duration 1.000000 [s] (100 loops)
2017-07-14 15:03:59,518 INFO    : Workload     : |  period    10000 [us], duty_cycle  10 %
2017-07-14 15:03:59,525 INFO    : Workload     : |  run_time   1000 [us], sleep_time   9000 [us]
2017-07-14 15:03:59,527 INFO    : Workload     : + phase_000002: duration 1.000000 [s] (20 loops)
2017-07-14 15:03:59,528 INFO    : Workload     : |  period    50000 [us

In [17]:
# Inspect the JSON file used to run the application
with open('./{}_00.json'.format(rtapp_name), 'r') as fh:
    rtapp_json = json.load(fh)
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)

2017-07-14 15:04:03,379 INFO    : root         : Generated RTApp JSON file:


{
    "global": {
        "calibration": 226, 
        "default_policy": "SCHED_OTHER", 
        "duration": -1, 
        "logdir": "/data/local/tmp/devlib-target"
    }, 
    "tasks": {
        "task_ramp": {
            "cpus": [
                4
            ], 
            "loop": 1, 
            "phases": {
                "p000001": {
                    "loop": 100, 
                    "run": 1000, 
                    "timer": {
                        "period": 10000, 
                        "ref": "task_ramp"
                    }
                }, 
                "p000002": {
                    "loop": 20, 
                    "run": 5000, 
                    "timer": {
                        "period": 50000, 
                        "ref": "task_ramp"
                    }
                }, 
                "p000003": {
                    "loop": 20, 
                    "run": 15000, 
                    "timer": {
                        "period": 50000, 
       

## 3) execution

In [18]:
res_dir = os.path.join(te.res_dir, rtapp_name)
nrg_report, plt, plt_file, trace_file = execute(te, rtapp, res_dir)

2017-07-14 15:04:19,393 INFO    : root         : # Create results folder for this execution
2017-07-14 15:04:19,513 INFO    : root         : # Setup FTrace
2017-07-14 15:04:21,960 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:04:22,083 DEBUG   : FtraceCollector : Trace CPUIdle states
2017-07-14 15:04:22,190 INFO    : root         : ### Start RTApp execution
2017-07-14 15:04:22,192 INFO    : Workload     : Workload execution START:
2017-07-14 15:04:22,196 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/example2_00.json 2>&1


2017-07-14 15:04:28,579 INFO    : root         : # Stop FTrace
2017-07-14 15:04:28,585 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:04:28,820 INFO    : root         : # Save FTrace: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example2/trace.dat
2017-07-14 15:04:30,677 INFO    : root         : # Save platform description: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example2/platform.json
2017-07-14 15:04:30,685 INFO    : root         : # Report collected data:
2017-07-14 15:04:30,688 INFO    : root         :    /data/lisa_remote/lisa/results/WlgenExample_hikey960/example2


total 4548
drwxrwxr-x 2 zhifei zhifei    4096 7月  14 15:04 .
drwxrwxr-x 4 zhifei zhifei    4096 7月  14 15:04 ..
-rw-r--r-- 1 zhifei zhifei    2131 7月  14 15:04 example2_00.json
-rw-rw-r-- 1 zhifei zhifei     170 7月  14 15:04 output.log
-rw-rw-r-- 1 zhifei zhifei    1528 7月  14 15:04 platform.json
-rw-r--r-- 1 zhifei zhifei   25084 7月  14 15:04 rt-app-task_ramp-0.log
-rw-r--r-- 1 zhifei zhifei 4608000 7月  14 15:04 trace.dat


## 4) trace inspection

In [19]:
!kernelshark {trace_file} 2>/dev/null

version = 6


# Custom RTApp connfiguration

In [20]:
# Support to configure and run RTApp based workloads
from wlgen import RTA

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp_name = 'example3'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp to use a custom JSON
rtapp.conf(

    # 1. generate a "custom" set of tasks
    kind = 'custom',
    
    # 2. define the "profile" of each task
    params = "../../../assets/mp3-short.json",
    
    # In this case only few values of the orignal JSON can be tuned:
    #    DURATION : maximum duration of the workload [s]
    #    PVALUE   : calibration value
    #    LOGDIR   : folder used for generated logs
    #    WORKDIR  : working directory on target
    
    # 3. defined a maximum duration for that workload
    duration = 5,
    
);

2017-07-14 15:04:46,659 INFO    : Workload     : Setup new workload example3
2017-07-14 15:04:46,776 INFO    : Workload     : Loading custom configuration:
2017-07-14 15:04:46,778 INFO    : Workload     :    ../../../assets/mp3-short.json


In [21]:
res_dir = os.path.join(te.res_dir, rtapp_name)
nrg_report, plt, plt_file, trace_file = execute(te, rtapp, res_dir)

2017-07-14 15:04:47,287 INFO    : root         : # Create results folder for this execution
2017-07-14 15:04:47,401 INFO    : root         : # Setup FTrace
2017-07-14 15:04:49,825 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:04:49,930 DEBUG   : FtraceCollector : Trace CPUIdle states
2017-07-14 15:04:50,017 INFO    : root         : ### Start RTApp execution
2017-07-14 15:04:50,025 INFO    : Workload     : Workload execution START:
2017-07-14 15:04:50,026 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/example3_00.json 2>&1


2017-07-14 15:04:55,968 INFO    : root         : # Stop FTrace
2017-07-14 15:04:55,977 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:04:56,182 INFO    : root         : # Save FTrace: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example3/trace.dat
2017-07-14 15:04:58,038 INFO    : root         : # Save platform description: /data/lisa_remote/lisa/results/WlgenExample_hikey960/example3/platform.json
2017-07-14 15:04:58,046 INFO    : root         : # Report collected data:
2017-07-14 15:04:58,052 INFO    : root         :    /data/lisa_remote/lisa/results/WlgenExample_hikey960/example3


total 4952
drwxrwxr-x 2 zhifei zhifei    4096 7月  14 15:04 .
drwxrwxr-x 5 zhifei zhifei    4096 7月  14 15:04 ..
-rw-r--r-- 1 zhifei zhifei    1334 7月  14 15:04 example3_00.json
-rw-r--r-- 1 zhifei zhifei   20746 7月  14 15:04 mp3-AudioOut-1.log
-rw-r--r-- 1 zhifei zhifei  103206 7月  14 15:04 mp3-AudioTick-0.log
-rw-r--r-- 1 zhifei zhifei   20870 7月  14 15:04 mp3-AudioTrack-2.log
-rw-r--r-- 1 zhifei zhifei   20869 7月  14 15:04 mp3-mp3.decoder-3.log
-rw-r--r-- 1 zhifei zhifei   20745 7月  14 15:04 mp3-OMXCall-4.log
-rw-rw-r-- 1 zhifei zhifei     726 7月  14 15:04 output.log
-rw-rw-r-- 1 zhifei zhifei    1528 7月  14 15:04 platform.json
-rw-r--r-- 1 zhifei zhifei 4845568 7月  14 15:04 trace.dat


In [22]:
# Inspect the JSON file used to run the application
with open('./{}_00.json'.format(rtapp_name), 'r') as fh:
    rtapp_json = json.load(fh)
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)

2017-07-14 15:04:58,197 INFO    : root         : Generated RTApp JSON file:


{
    "global": {
        "calibration": 226, 
        "default_policy": "SCHED_OTHER", 
        "duration": 5, 
        "frag": 1, 
        "ftrace": false, 
        "gnuplot": false, 
        "lock_pages": true, 
        "log_basename": "mp3", 
        "logdir": "/data/local/tmp/devlib-target"
    }, 
    "tasks": {
        "AudioOut": {
            "loop": -1, 
            "priority": -19, 
            "resume": "AudioTrack", 
            "run": 4725, 
            "suspend": "AudioOut"
        }, 
        "AudioTick": {
            "cpus": [
                0
            ], 
            "loop": -1, 
            "phases": {
                "p1": {
                    "loop": 1, 
                    "resume": "AudioOut", 
                    "timer": {
                        "period": 6000, 
                        "ref": "tick"
                    }
                }, 
                "p2": {
                    "loop": 4, 
                    "timer": {
                        "per

In [23]:
!kernelshark {trace_file} 2>/dev/null

version = 6


# Running Hackbench

## 1) creation and configuration

In [24]:
# Support to configure and run RTApp based workloads
from wlgen import PerfMessaging

# Create a "perf bench sched messages" (i.e. hackbench) workload
perf_name = 'hackbench'
perf = PerfMessaging(target, perf_name)

perf.conf(group=1, loop=100, pipe=True, thread=True)


2017-07-14 15:05:01,585 INFO    : Workload     : Setup new workload hackbench
2017-07-14 15:05:01,680 DEBUG   : perf_bench   :      PerfBench - Command line: /data/local/tmp/bin/perf bench sched messaging --pipe --thread --group 1 --loop 100


'hackbench_00'

## 2) execution

In [25]:
res_dir = os.path.join(te.res_dir, perf_name)
nrg_report, plt, plt_file, trace_file = execute(te, perf, res_dir)

2017-07-14 15:05:01,747 INFO    : root         : # Create results folder for this execution
2017-07-14 15:05:01,866 INFO    : root         : # Setup FTrace
2017-07-14 15:05:04,270 DEBUG   : FtraceCollector : Trace CPUFreq frequencies
2017-07-14 15:05:04,365 DEBUG   : FtraceCollector : Trace CPUIdle states
2017-07-14 15:05:04,470 INFO    : root         : ### Start RTApp execution
2017-07-14 15:05:04,473 INFO    : Workload     : Workload execution START:
2017-07-14 15:05:04,474 INFO    : Workload     :    /data/local/tmp/bin/perf bench sched messaging --pipe --thread --group 1 --loop 100
2017-07-14 15:05:04,626 DEBUG   : perf_bench   :      PerfBench - Saving output on [/data/lisa_remote/lisa/results/WlgenExample_hikey960/hackbench/output.log]...
2017-07-14 15:05:04,629 INFO    : perf_bench   :      PerfBench - Completion time: 0.090000, Performance 11.111111
2017-07-14 15:05:04,633 DEBUG   : perf_bench   :      PerfBench - Saving performance into [/data/lisa_remote/lisa/results/WlgenExa

2017-07-14 15:05:04,925 INFO    : root         : # Save FTrace: /data/lisa_remote/lisa/results/WlgenExample_hikey960/hackbench/trace.dat
2017-07-14 15:05:06,934 INFO    : root         : # Save platform description: /data/lisa_remote/lisa/results/WlgenExample_hikey960/hackbench/platform.json
2017-07-14 15:05:06,953 INFO    : root         : # Report collected data:
2017-07-14 15:05:06,959 INFO    : root         :    /data/lisa_remote/lisa/results/WlgenExample_hikey960/hackbench


total 4988
drwxrwxr-x 2 zhifei zhifei    4096 7月  14 15:05 .
drwxrwxr-x 6 zhifei zhifei    4096 7月  14 15:05 ..
-rw-rw-r-- 1 zhifei zhifei     142 7月  14 15:05 output.log
-rw-rw-r-- 1 zhifei zhifei      60 7月  14 15:05 performance.json
-rw-rw-r-- 1 zhifei zhifei    1528 7月  14 15:05 platform.json
-rw-r--r-- 1 zhifei zhifei 5087232 7月  14 15:05 trace.dat


## 3) explore the performance report

In [26]:
# Inspect the generated performance report
perf_file = os.path.join(te.res_dir, perf_name, 'performance.json')
with open(perf_file, 'r') as fh:
    perf_json = json.load(fh)
logging.info('Generated performance JSON file:')
print json.dumps(perf_json, indent=4, sort_keys=True)

2017-07-14 15:05:07,116 INFO    : root         : Generated performance JSON file:


{
    "ctime": 0.09, 
    "performance": 11.11111111111111
}


## 4) trace inspection 

In [27]:
!kernelshark {trace_file} 2>/dev/null

version = 6
